# Ataques ao RSA

O criptosistema RSA baseia-se na dificuldade em determinar $p$ e $q$ dado um inteiro $N$, tal que $N=p*q$ com $p$ e $q$ primos.

Pretende-se implementar os ataques à inversão da chave pública e inversão do criptograma, usando redução de bases.  Verificando para que gama de valores dos parâmetros $N,q$ esses ataques são viáveis.

## Teorema (CopperSmith, Howgrave-Graham,May)

Dado um inteiro $N>1$ e um polinómio mónico $f \in \mathbb{Z}[x]$, para todo o parâmetro racional $0<\beta\le 1$, é possivel determinar em tempo polinomial, todos os inteiros $w$ tais que:
$|w| \le N^{\beta ^2 /d}$ e $mdc(f(w),N) \ge N^{\beta}$.

A importância do algoritmo de CopperSmith e, de forma geral, dos algoritmos de redução de bases em reticulados, provém da sua aplicabilidade em ataques a algumas das técnicas mais usadas, nomeadamente ao RSA.

## Invertibilidade do criptograma RSA
No início do uso do RSA além de não se usar qualquer forma de *padding* aleatória, também prevalecia o mau hábito de usar como chave pública um número primo pequeno, sendo frequente usar-se, para cifrar, um expoente público $d = 3$. Consequentemente, o criptograma relacionava-se com o *plaintext* numa relação da forma:
$$c \equiv m^d mod N,$$
sendo N o módulo RSA e $c,m \in \mathbb{Z}_{N}^{*}$ o ciptograma e o *plaintext*, respetivamente.

Um **ataque por inversão de criptograma** resolve o problema: $$N,d,c\leadsto m,$$
sem passar pelo conhecimento da chave pública.
Para usar o **algoritmo de CopperSmith** vamos alterá-lo ligeiramente. Considere-se: $$m = m' + w$$ e o seguinte problema:
$$N,d,c,m' \leadsto w: (w+m')^d \equiv c \ mod N.$$

Isto é, conhecendo uma parte do *plaintext*, procura-se determinar a diferença: $w = m -m'$, entre o *plaintext* que se pretende conhcer e a parte que já é conhecida.

Assim o algoritmo de CopperSmith é, assim, aplicado ao polinómio: $$f(x) \equiv (x+m')^d -c$$
e o que se pretende determinar é a raiz modular deste $f$, i.e., pretende-se clacular $w$ que verifique: $$f(w) \equiv 0 \ mod \ N.$$

As condições do algoritmo exigem que $w$ seja limitada a um valor $X \le N^{\beta^2/d}.$

Usando a versão do algoritmo $\beta = 1$ e considerando que o ataque se faz em relação à má chave pública, $d = 3$, então será $X \le N^{1/3}$. Em termos de tamanh, X, que é o tamanho da icógnita w, tem um terço dos *bits* de N.

Com estes parâmetros, o algoritmo de CopperSmith calcula as raizes modulares do polinómio $$f(x) \equiv (x+m')^d-c.$$
A raiz $w$, que verifica $f(w) \equiv 0 \ mod \ N$, adicionada à parte conhecida $m'$, produz todo o *plaintext* m.

## Factorização do módulo RSA
Uma das facetas mais úteis do algoritmo de Coppersmith é a sua capacidade de resolver equações modulares, $$f(w) \equiv mod \ B,$$ em que o módulo $B$ é um divisor desconhecido de um inteiro conhecido $N$.

Normalmente, quando $N$ é um módulo RSA ele é o produto de dois primos desconhecidos (os únicos divisores de $N$) $$N = p * q $$ e o algoritmo vai permitir reslver uma equação da forma $$f(w) \equiv 0 mod \ p$$ sem passar pelo cálculo prévio do factor $p$.

As condições do algoritmo exigem que um divisor B qualquer de N verifique: $$B \ge N^{\beta}$$ e detrrmina raizes $w$ limitadas, em valor absoluta, a um valor $X \le N ^{\beta^2/d}$.
Seja p o maior factor de N; como $N = p * q$ então temos de ter $p \ge \sqrt(N)$. Isto justifica que se use, no algoritmo de CopperSmith, um valor de $\beta $ que seja $1/2$ ou ligeiramnete superior
$$\beta = 1/2 + 0(1)$$ para solucionar o maior divisor de N que seja diferente do próprio N. O respetivo valor de X será agora limitado a $$x \le (N^{1/D})^{1/4}.$$
Vamos escolher uma função $f(x)$ do primeiro grau da seguinte forma: $$f(x) \equiv x + k,$$ sendo $k$ um parâmetro conhecido mais ainda não identificado e usemos o algorimo de CopperSmith para determinar todos os inteiros $w$ que são raizes de $f(.) \ mod \ p$.

In [ ]:
class RSA:
    def __init__(self, l):
        self.l = int(l)
        self.keygen()

    def keygen(self):
        r = random_prime(2^(floor(l/2)))
        p = random_prime(2*r+1)
        q = self.p*self.r
        m = (self.p-1)*(self.r-1)
        k = randint(2,m)
        while gcd(m,k)!=1:
            k = randint(2,m)
        s = power_mod(k,-1,m)
        self.pubkey = (q,k)
        self.privkey = s

    def cifra(self,a):
        q,k = self.pubkey
        c = power_mod(a,k,q)
        return c

    def decifra(self, c):
        s = self.privkey
        q,_=self.pubkey
        z = power_mod(c,s,q)
        return z

    def assina(self,m):
        sig = power_mod(m,self.privkey,self.pubkey[0])
        return sig

    def verifica(self,m, sig):
        m_k = power_mod(sig,self.pubkey[1],self.pubkey[0])
        return (m_k == m)

def small_roots2(f, X=None):
    d = f.degree()
    K = f.base_ring()
    M = K.characteristic()
    f.change_ring(ZZ)
    if X is None:
        X =  M.nth_root(d*(d+1)/2)
    A = Matrix(ZZ,d+1,d+1)
    for j in range(d):
        A[j,j] = M*X^j
    for j in range(d+1):
        A[d,j] = ZZ(f[j])*X^j
    A = A.LLL()
    x = ZZ['x'].gen(0)
    g = 0
    for j in range(d+1):
        g+= A[0,j]/X^j * x^j
    return map(lambda (x,y):(K(x),y), g.roots())

class CopperSmith:
    def __init__(self, rsa):
        if(isinstance(rsa,RSA)):
            self.rsa = rsa
        else:
            raise('rsa parameter must be an object of class RSA')
    
    def reverse_pub_key(self, beta):
        N = Integers(self.rsa.pubkey[0])
        hbits = self.rsa.l//4 - 2
        hidden_bits = ZZ.random_element(2^(hbits-1), 2^hbits-1)
        leaked_bits = self.rsa.p - hidden_bits
        
        R.<x> = PolynomialRing(N, implementation='NTL')
        f = x + leaked_bits
        
        X = floor(self.rsa.pubkey[0] ^ (beta**2)) 
        #f is of degree one, so d=1
        
        rts = f.small_roots(beta=beta, X=X)
        
        if len(rts)>0:
            recovered_hidden_b = rts[0]
            recovered_p = lift(leaked_bits+recovered_hidden_b)
            print(recovered_p == self.rsa.p)
            print(self.rsa.pubkey[0] // recovered_p == self.rsa.r)
    
    
    def reverse_pub_key_lattice(self, beta):
        N = Integers(self.rsa.pubkey[0])
        hbits = self.rsa.l//4 - 2
        hidden_bits = ZZ.random_element(2^(hbits-1), 2^hbits-1)
        leaked_bits = self.rsa.p - hidden_bits
        
        R.<x> = PolynomialRing(N, implementation='NTL')
        f = x + leaked_bits
        X = floor(self.rsa.pubkey[0] ^ (beta**2)) 
        #f is of degree one, so d=1
        
        roots = map(QQ, [leaked_bits**i for i in range(2)])
        LW = matrix(QQ, 2, 1, roots)
        G=identity_matrix(QQ, 2).augment(LW)
        print(G)
        print(G.LLL())
        print(hidden_bits)
        
        rts = f.small_roots(beta=beta, X=X)
        if len(rts)>0:
            recovered_hidden_b = rts[0]
            recovered_p = lift(leaked_bits+recovered_hidden_b)
            print(recovered_p == self.rsa.p)
            print(self.rsa.pubkey[0] // recovered_p == self.rsa.r)
            
    def reverse_cript(self,c,m_2):
        N = Integers(self.rsa.pubkey[0])
        d = 3
        beta = 1

        R.<x> = PolynomialRing(N, implementation='NTL')
        f = (x + m_2)^d - c

        X = floor(self.rsa.pubkey[0] ^ (beta**2/3))
        w = small_roots2(f
        if len(rts)>0:
            if len(m)<680:
                m = w + m_2
        return w
        

In [ ]:
rsa = RSA(128)
copper = CopperSmith(rsa)
copper.reverse_pub_key_lattice(1/2)